In [1]:
# !pip install imblearn

import os

import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE


In [2]:
abt = pd.read_csv("../Resources/analytical_base_table.csv")
abt.head()

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,France,Female,42,2,0.00,1,1,101348.88,1
1,Spain,Female,41,1,83807.86,1,1,112542.58,0
2,France,Female,42,8,159660.80,3,0,113931.57,1
3,France,Female,39,1,0.00,2,0,93826.63,0
4,Spain,Female,43,2,125510.82,1,1,79084.10,0


## Models Training

## Let's start by splitting our dataframe into separate objects:

y for the target varibale

X for the input features

In [3]:
#Separate dataframe into separate object

# Object for target variable
y = abt.Exited

# object for input features
X = abt.drop(['Exited'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)


(10000, 8) (10000,)


In [4]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'IsActiveMember',
 'EstimatedSalary']

In [5]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Geography', 'Gender']

In [6]:
def class_count(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    abt2 = pd.DataFrame(np.array(kv).T, columns=['Exited','Count'])
    abt2['Count'] = abt2['Count'].astype('int64')
    abt2['%'] = round(abt2['Count'] / a.shape[0] * 100, 2)
    return abt2.sort_values('Count',ascending=False)

In [7]:
class_count(y)


,Exited,Count,%
1,0,7963,79.63
0,1,2037,20.37


## Create a Train Test Split

We will continue with splitting our data into separate training and test sets.

30% of observations will be set aside for the test set

the rest, 70%, will be used as the training set

In [8]:
random_state = 8

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=abt.Exited)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7000 entries, 6347 to 2994
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Geography        7000 non-null   object 
 1   Gender           7000 non-null   object 
 2   Age              7000 non-null   int64  
 3   Tenure           7000 non-null   int64  
 4   Balance          7000 non-null   float64
 5   NumOfProducts    7000 non-null   int64  
 6   IsActiveMember   7000 non-null   int64  
 7   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(4), object(2)
memory usage: 492.2+ KB


## Pre-processing Pipeline
 
 ##Scale numerical data and encode categorical data
Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder Create lists of indexes from the list of column namesNeed to be numeric not string to specify columns name in column transformer

In [10]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)  

[2, 3, 4, 5, 6, 7]


In [11]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features) 

[0, 1]


In [12]:
# Define column transformer
# Need to be numeric not string to specify columns name 
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(), cat_features)
)
preprocess

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [2, 3, 4, 5, 6, 7]),
                                ('onehotencoder', OneHotEncoder(), [0, 1])])

In [13]:
# Import classifier
from sklearn.linear_model import LogisticRegression 

# Define model with pipeline
model = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  LogisticRegression(random_state=random_state))

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [2, 3, 4, 5, 6, 7]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(), [0, 1])])),
                ('smote', SMOTE(random_state=8)),
                ('logisticregression', LogisticRegression(random_state=8))])

In [14]:
 # Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
lr_param_grid = {
    'logisticregression__C' : [0.01, 0.05, 0.1, 0.5, 1, 5],
    'logisticregression__solver' : ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
}

lr_grid = GridSearchCV(model, lr_param_grid, verbose=3, cv= 5, scoring='accuracy')

In [15]:
X_train = X_train.values
X_test = X_test.values

In [16]:
lr_grid.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[CV 1/5] END logisticregression__C=0.01, logisticregression__solver=liblinear;, score=0.678 total time=   0.0s
[CV 2/5] END logisticregression__C=0.01, logisticregression__solver=liblinear;, score=0.649 total time=   0.0s
[CV 3/5] END logisticregression__C=0.01, logisticregression__solver=liblinear;, score=0.694 total time=   0.0s
[CV 4/5] END logisticregression__C=0.01, logisticregression__solver=liblinear;, score=0.669 total time=   0.0s
[CV 5/5] END logisticregression__C=0.01, logisticregression__solver=liblinear;, score=0.690 total time=   0.0s
[CV 1/5] END logisticregression__C=0.01, logisticregression__solver=newton-cg;, score=0.683 total time=   0.0s
[CV 2/5] END logisticregression__C=0.01, logisticregression__solver=newton-cg;, score=0.650 total time=   0.0s
[CV 3/5] END logisticregression__C=0.01, logisticregression__solver=newton-cg;, score=0.699 total time=   0.0s
[CV 4/5] END logisticregression__C=0.01, logisticregression__solver=newton-cg;, score=0.669 total time=   0.0s
[

s:\projects\vlu\Số hóa và Quản lý thông tin số\example_project\Bank-Churn-Prediction\myvenv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
s:\projects\vlu\Số hóa và Quản lý thông tin số\example_project\Bank-Churn-Prediction\myvenv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END logisticregression__C=5, logisticregression__solver=saga;, score=0.699 total time=   0.0s
[CV 3/5] END logisticregression__C=5, logisticregression__solver=saga;, score=0.724 total time=   0.0s


s:\projects\vlu\Số hóa và Quản lý thông tin số\example_project\Bank-Churn-Prediction\myvenv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
s:\projects\vlu\Số hóa và Quản lý thông tin số\example_project\Bank-Churn-Prediction\myvenv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END logisticregression__C=5, logisticregression__solver=saga;, score=0.698 total time=   0.0s
[CV 5/5] END logisticregression__C=5, logisticregression__solver=saga;, score=0.706 total time=   0.0s


s:\projects\vlu\Số hóa và Quản lý thông tin số\example_project\Bank-Churn-Prediction\myvenv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [2, 3,
                                                                          4, 5,
                                                                          6,
                                                                          7]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         [0,
                                                                          1])])),
                                       ('smote', SMOTE(random_state=8)),
                                       ('logisticregression',
                                        LogisticRegression(random_state=8))]),
             param_grid={'logisticregression__C': [0.01, 0.05, 0.1, 0.5, 1, 5],
                         'logisticregression__solver': ['liblinear',
                                                        'newton-cg', 'lbfgs',
                                                        'sag', 'saga']},
             scoring='accuracy', verbose=3)

In [17]:
print(lr_grid.best_params_)

{'logisticregression__C': 5, 'logisticregression__solver': 'liblinear'}


In [18]:
print(lr_grid.best_score_)

0.7088571428571429


In [19]:
print(f"Training Data Score: {lr_grid.score(X_train, y_train)}")
print(f"Testing Data Score: {lr_grid.score(X_test, y_test)}")

Training Data Score: 0.7097142857142857
Testing Data Score: 0.7226666666666667


In [20]:
predictions = lr_grid.predict(X_test)
print(f"First 8 Predictions:   {predictions[:8]}")
print(f"First 8 Actual labels: {y_test[:8].tolist()}")

First 8 Predictions:   [1 0 1 1 1 0 0 0]
First 8 Actual labels: [1, 0, 0, 1, 1, 0, 0, 0]


In [21]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1,1
1,0,0
2,1,0
3,1,1
4,1,1
...,...,...
2995,0,0
2996,0,0
2997,0,0
2998,0,0


In [22]:
cm = confusion_matrix(y_test, predictions)
print(cm)

[[1728  661]
 [ 171  440]]


In [23]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.72 0.28]
 [0.28 0.72]]


In [24]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.72      0.81      2389
           1       0.40      0.72      0.51       611

    accuracy                           0.72      3000
   macro avg       0.65      0.72      0.66      3000
weighted avg       0.81      0.72      0.75      3000



In [25]:
pred = lr_grid.predict(X_test[:1])

In [26]:
print(f"Predicted classes: {pred}")
print(f"Actual Labels: {list(y_test[:1])}")

Predicted classes: [1]
Actual Labels: [1]


In [27]:
import joblib

filename = '../models/nate_logistic_regression.sav'
joblib.dump(lr_grid, filename)

['../models/nate_logistic_regression.sav']

In [28]:
lr_model = joblib.load(filename)
print(lr_model.score(X_test, y_test))

0.7226666666666667


### Predict class for new data

In [29]:
# Let's use the first X_test record as new data
X_test[:1]

array([['France', 'Male', 50, 4, 165438.26, 1, 0, 120770.75]],
      dtype=object)

In [30]:
pred_new = lr_grid.predict(X_test[:1])

In [31]:
print(f"Predicted classes: {pred_new}")
print(f"Actual Labels: {list(y_test[:1])}")

Predicted classes: [1]
Actual Labels: [1]


In [32]:
X_test[:1]


array([['France', 'Male', 50, 4, 165438.26, 1, 0, 120770.75]],
      dtype=object)

In [33]:
pred_new1 = lr_grid.predict(X_test[:1])
pred_new1

array([1], dtype=int64)